### 데이터 전처리
- spaCy 라이브러리 : spaCy 라이브러리: 문장의 토큰화(tokenization), 태깅(tagging) 등의 전처리 기능을 위한 라이브러리

In [1]:
# 영어와 독일어 전처리 모듈 설치
%%capture
!python -m spacy download en
!python -m spacy download de

In [3]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)
spacy_de = spacy.load('de_core_news_sm') # 독일어 토큰화(tokenization)

In [5]:
# 간단히 토큰화(tokenization) 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate Student.")

for i, token in enumerate(tokenized):
  print(f"인덱스 {i}: {token}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: Student
인덱스 5: .


In [6]:
# tokenizer를 가볍게 다양하게 사용해보기
tokenized = spacy_en.tokenizer("I am a graduate student.  But I do not well at English.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .
인덱스 6:  
인덱스 7: But
인덱스 8: I
인덱스 9: do
인덱스 10: not
인덱스 11: well
인덱스 12: at
인덱스 13: English
인덱스 14: .


### 독일어를 영어로 번역하는 task

In [9]:
#독일어를 인코더에 들어가는 input data로 쓸것임.
#seq2seq 기본 논문에서는 입력문장을 넣을때는 토큰 순서를 바꾸어서 넣는다고 하였음

# 독일어(Deutsch) 문장을 토큰화한 뒤에 순서를 뒤집는 함수
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)][::-1]

# 영어(English) 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]


tokenized = tokenize_en("I love a chicken.")
print(tokenized) # 그냥 문장이 들어오면, 단어 단위로 쪼개준다고 생각하면 편함.

['I', 'love', 'a', 'chicken', '.']


In [10]:
# 최신 torchtext에서는 Field, BucketIterator가 torchtext.legacy.data로 이전됐다고 함.
# 그래서 옛날 버전으로 회귀하기
# pip => python package 관리 도구 중
!pip install -U torchtext==0.6

# 만약에 런타임 재부팅 하라는 메시지가 나오면, 다시 처음부터 위에서부터 실행

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2



###Field의 주요 기능과 속성은 다음과 같습니다:

Field 객체는 PyTorch의 torchtext 라이브러리에서 제공하는 클래스로, 텍스트 데이터의 전처리와 변환을 위한 설정과 도구를 제공합니다. 특히, 텍스트 데이터를 로드하고, 토큰화하며, 숫자로 변환하고, 패딩을 추가하는 등의 작업을 수행할 때 사용됩니다.

- 토큰화: 주어진 텍스트를 토큰 단위로 분리하는 작업을 수행합니다. 사용자는 원하는 토큰화 함수를 지정할 수 있습니다.

- 어휘 구축: 데이터셋에서 모든 유니크한 토큰을 수집하여 어휘를 구축합니다. 이 어휘는 텍스트 토큰을 숫자로 변환하는 데 사용됩니다.

- 숫자 변환: 텍스트 토큰을 해당하는 숫자 ID로 변환합니다. 이 변환은 모델 학습에 필요한 숫자 텐서로의 변환을 위해 수행됩니다.

- 패딩: 모든 문장이 동일한 길이를 갖도록 패딩 토큰을 추가합니다. 이는 배치 처리를 위해 필요합니다.

- 기타 설정: 예를 들어, 문장의 시작과 끝에 특별한 토큰을 추가하거나, 모든 문자를 소문자로 변환하는 등의 작업을 수행할 수 있습니다.

###Field 객체를 생성할 때 주로 사용되는 인자들은 다음과 같습니다:

- tokenize: 텍스트를 토큰화하는 함수를 지정합니다.
- init_token: 각 문장의 시작에 추가될 토큰을 지정합니다.
- eos_token: 각 문장의 끝에 추가될 토큰을 지정합니다.
- lower: 모든 문자를 소문자로 변환할지 여부를 지정합니다.
- pad_token: 패딩에 사용될 토큰을 지정합니다.
- unk_token: 어휘에 없는 토큰을 대체할 토큰을 지정합니다.

이러한 설정을 통해 Field 객체는 텍스트 데이터를 모델에 적합한 형식으로 쉽게 변환하는 데 도움을 줍니다.




### BucketIterator의 주요 기능은 다음과 같습니다:
BucketIterator는 PyTorch의 torchtext 라이브러리에서 제공하는 클래스로, 텍스트 데이터의 배치를 생성할 때 사용됩니다. 특히, BucketIterator의 주요 목적은 텍스트 데이터의 특성을 고려하여 효율적인 배치 생성을 도와주는 것입니다.

텍스트 데이터의 문장들은 다양한 길이를 가질 수 있습니다. 따라서, 모든 문장을 동일한 길이로 만들기 위해 패딩을 추가해야 합니다. 그런데 임의로 문장들을 배치로 묶으면, 매우 짧은 문장과 매우 긴 문장이 함께 배치될 수 있어 많은 패딩이 필요하게 됩니다. 이는 계산적으로 비효율적입니다.

비슷한 길이의 문장 그룹화: BucketIterator는 길이가 비슷한 문장들을 함께 그룹화하여, 각 배치마다 필요한 패딩의 양을 최소화합니다.

동적 패딩: 각 배치마다 필요한 만큼만 패딩을 추가하여, 계산적인 낭비를 줄입니다.

배치 생성: 지정된 배치 크기에 따라 데이터를 배치로 분할합니다.

### BucketIterator를 사용하여 배치를 생성할 때 주로 사용되는 인자들은 다음과 같습니다:

data: 배치를 생성할 데이터셋을 지정합니다.
batch_size: 각 배치에 포함될 문장의 수를 지정합니다.
device: 텐서를 어느 디바이스(CPU 또는 GPU)에 할당할지 지정합니다.
sort_key: 문장들을 어떤 기준으로 정렬할지 지정하는 함수입니다. 대부분의 경우, 문장의 길이를 기준으로 합니다.
shuffle: 각 epoch마다 데이터를 섞을지 여부를 지정합니다.
이러한 기능을 통해, BucketIterator는 텍스트 데이터의 특성을 고려하여 효율적인 배치 생성을 도와줍니다.







In [12]:
from torchtext.data import Field, BucketIterator


# init_token:  토큰
# eos_token:  토큰
# lower는 그냥 대문자를 무시할 것인지 설정(True면 전부 다 소문자로 처리)
# I love you = i love you (대문자가 없어짐) => 토큰 수를 줄일 수 있어요. (|V|를 줄일 수 있음)
SRC = Field(tokenize=tokenize_de, init_token="", eos_token="", lower=True)
TRG = Field(tokenize=tokenize_en, init_token="", eos_token="", lower=True)

In [13]:
# Python 라이브러리는 PIP로 설치해서 사용가능
# 설치된 라이브러리는 /usr/local/lib/python3.10/dist-packages에 존재

!ls /usr/local/lib/python3.10/dist-packages/torchtext/datasets
# !cat /usr/local/lib/python3.10/dist-packages/torchtext/datasets/translation.py (torchtext 0.6.0)
# !cat /usr/local/lib/python3.10/dist-packages/torchtext/datasets/multi30k

babi.py		      nli.py		   text_classification.py
imdb.py		      __pycache__	   translation.py
__init__.py	      sequence_tagging.py  trec.py
language_modeling.py  sst.py		   unsupervised_learning.py


In [14]:
!pip show torchtext

Name: torchtext
Version: 0.6.0
Summary: Text utilities and datasets for PyTorch
Home-page: https://github.com/pytorch/text
Author: PyTorch core devs and James Bradbury
Author-email: jekbradbury@gmail.com
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, requests, sentencepiece, six, torch, tqdm
Required-by: 


In [15]:
import torchtext

# print(torchtext.datasets.Multi30k.urls)

# 현재 다운로드 링크가 동작하지 않으므로, 동작되는 URL로 변경
torchtext.datasets.Multi30k.urls = [
    r"https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz",
    r"https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz",
    r"https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz",
]

print(torchtext.datasets.Multi30k.urls)

['https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz', 'https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz', 'https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz']


In [16]:
from torchtext.datasets import Multi30k

# 총 30,000개의 (독일어, 영어) 쌍으로 구성된 데이터 세트
train_dataset, valid_dataset = Multi30k.splits(exts=(".de", ".en"),
                                               fields=(SRC, TRG),
                                               root='.custom_data',
                                               train="train",
                                               validation="val",
                                               test=None)

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 12.2MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 2.94MB/s]


downloading mmt16_task1_test.tar.gz


mmt16_task1_test.tar.gz: 100%|██████████| 67.1k/67.1k [00:00<00:00, 3.24MB/s]


In [17]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개


In [18]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[30])['src']) # 순서를 바꿨기 때문에, 온점이 제일 첫째로 나옴.
print(vars(train_dataset.examples[30])['trg']) # 정상적인 순서

['.', 'steht', 'urinal', 'einem', 'an', 'kaffee', 'tasse', 'einer', 'mit', 'der', ',', 'mann', 'ein']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [19]:
# 최소 두 번 이상 등장한 단어만 기록
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7852
len(TRG): 5892


In [21]:
# 그러면, 한 번만 등장한 단어들은 컴퓨터에서 어떻게 처리되지?
#  토큰으로 처리가 됨.

# 자연어 처리 분야에서 "특수 토큰"은 보통 4가지가 존재
"""
1)  <sos> 문장의 시작을 알림
2)  <eos> 문장의 끝을 알림
3)  <unk> 처음 보는 단어를 알림
4)  <pad> 짧은 문장의 경우 뒤쪽을 으로 채움
"""

'\n1)  <sos> 문장의 시작을 알림\n2)  <eos> 문장의 끝을 알림\n3)  <unk> 처음 보는 단어를 알림\n4)  <pad> 짧은 문장의 경우 뒤쪽을 으로 채움\n'

In [22]:
# string to index (stoi)
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi[""]) # : 2
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
4111
1751


In [23]:
# 어떤 배치에서는 단어가 많은 문장만 나오다가
# 갑자기 어떤 배치에서는 단어가 적은 문장이 나오다가...
# 학습이 불안정해질 수 있음

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset),
    batch_size=BATCH_SIZE,
    device=device)

In [24]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"첫 번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[0]):
        print(f"인덱스 {i}: {src[i][0].item()}")

    # 첫 번째 배치만 확인
    break


첫 번째 배치 크기: torch.Size([31, 128])
인덱스 0: 2
인덱스 1: 3
인덱스 2: 188
인덱스 3: 3280
인덱스 4: 305
인덱스 5: 6666
인덱스 6: 105
인덱스 7: 101
인덱스 8: 5
인덱스 9: 10
인덱스 10: 12
인덱스 11: 4
인덱스 12: 2
인덱스 13: 1
인덱스 14: 1
인덱스 15: 1
인덱스 16: 1
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1
인덱스 27: 1
인덱스 28: 1
인덱스 29: 1
인덱스 30: 1


In [25]:
# 첫 번째 배치 크기: torch.Size([27, 128])

# 현재 배치에 포함된 문장의 수가 128개 (batch_size = 128)
# 이때 27이 무엇이냐면? 128개 중에서 가장 긴 문장의 토큰 개수

"""
batch_size = 2

"I love you"
"I like your football team"

=> 학습 데이터 상에서는
[, "i", "love", "you", , , ]
[, "i", "like", "your", "football", "team", ]

"""

'\nbatch_size = 2\n\n"I love you"\n"I like your football team"\n\n=> 학습 데이터 상에서는\n[, "i", "love", "you", , , ]\n[, "i", "like", "your", "football", "team", ]\n\n'

### 인코더

In [36]:
import torch.nn as nn

# 인코더(Encoder) 아키텍처 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding)을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더는 소스 문장을 입력으로 받아 문맥 벡터(context vector)를 반환
    def forward(self, src):
        # src = []"I love you dady"
        # src = " I love you    "
        # src = "2 328 594 291 0 1 1 3"
        # src: [단어 개수, 배치 크기]: 각 단어의 인덱스(index) 정보
        # 토큰별로 |V| 크기의 one-hot 벡터
        # src = [10,000차원 벡터, 10,000차원 벡터, 10,000차원 벡터, 10,000차원 벡터, 10,000차원 벡터, 10000차원 벡터, 10,000차원 벡터]
        embedded = self.dropout(self.embedding(src))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원] (임베딩 차원이 512라면)
        # src = [512차원 벡터, 512차원 벡터, 512차원 벡터, 512차원 벡터 ,512차원 벡터, 512차원 벡터, 512차원 벡터]

        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs: [단어 개수, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보

        # hidden, cell은 가장 마지막 토큰을 넣었을 때 나온 hidden states가 됨.
        # => 이것을 기계 번역에서 디코더에 들어가는 초기 입력

        # 문맥 벡터(context vector) 반환
        return hidden, cell

### 디코더

In [37]:
# 디코더(Decoder) 아키텍처 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding) 말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # FC 레이어 (인코더와 구조적으로 다른 부분)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

   # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환
    def forward(self, input, hidden, cell):
        # seq2seq는 항상 (1) 이전에 출력한 단어와 (2) 이전까지의 정보를 담은 hidden state가 입력으로 들어감
        # 그러므로, 이전에 출력했던 모든 단어에 대한 것은 필요 X
        # 이 디코더를 가 나올 때까지 여러 번 forward해서 전체 문자열을 출력할 것.

        # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
        # hidden: [레이어 개수, 배치 크기, 히든 차원]
        # cell = context: [레이어 개수, 배치 크기, 히든 차원]
        input = input.unsqueeze(0)

        # PyTorch에서 squeeze()는 차원 축소 (불필요한 차원 제거)
        # PyTorch에서 unsqueeze()는 차원 늘리기 (그냥 크기가 1인 axis 추가)

        # input: [단어 개수 = 1, 배치 크기]

        embedded = self.dropout(self.embedding(input))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output: [단어 개수 = 1, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보

        # 단어 개수는 어차피 1개이므로 차원 제거
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [배치 크기, 출력 차원]

        # (현재 출력 단어, 현재까지의 모든 단어의 정보, 현재까지의 모든 단어의 정보)
        return prediction, hidden, cell

In [38]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]
        # 먼저 인코더를 거쳐 문맥 벡터(context vector)를 추출
        hidden, cell = self.encoder(src)

        # 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫 번째 입력은 항상  토큰
        input = trg[0, :]

       # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            # 가 나올 때까지 재귀적으로 사용
            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output # FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # 현재 구조상, 만약에 디코더 입장에서 이전 단어가 틀리면, 앞으로의 단어도 계속 틀릴 가능성이 매우 높음
            #  => 이 상태로 그대로 학습하면, 너무 많이 틀려서 loss가 과도하게 커짐
            # "그래서, 강제로 특정 확률에 해당할 때는 "앞으로의 단어"를 고정해서 정답으로 알려주는 방법을 사용" (teacher forcing 방법)

            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio # random.random()은 [0, 1] uniform하게 값을 뽑음 => 평균 0.5

            # 마치 선생님(teacher)이 현재 단어는 "love"라고 출력해야 하는데, "hate"이라고 출력했네? 이 부분은 고쳐줄게.
            # => 정답 자체를 교정(recursive 도중에)
            # 만약 teacher_force의 값이 True라면, 강제로 "정답"을 다음 입력으로 넣어주는 겁니다. (강제로 정답을 맞추도록 forcing)
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기

        return outputs

In [39]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256 # 각 토큰(단어)가 표현되는 차원
DECODER_EMBED_DIM = 256 # 각 토큰(단어)가 표현되는 차원
HIDDEN_DIM = 512 # LSTM 내부에서 사용되는 hidden vector의 차원
N_LAYERS = 2 # LSTM을 얼마나 높게 쌓을 것인지
ENC_DROPOUT_RATIO = 0.5 # dropout => 모델 성능 향상 기법 중 하나
DEC_DROPOUT_RATIO = 0.5 # dropout => 모델 성능 향상 기법 중 하나

In [40]:
# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

In [41]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7852, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5892, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5892, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

### Optimizer 설정

In [42]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)##

### 학습 & 평가

In [43]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기, 출력 차원]
        output_dim = output.shape[-1]

        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어의 개수 - 1) * batch size, output dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어의 개수 - 1) * batch size]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산
       # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [44]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    # torch.no_grad() => 이 안에 들어가는 모델 및 입력에 대해서는 "기울기 추적"을 하지 않음.
    # (inference = 학습된 모델을 추론할 때 / 쓸 때만 사용)
    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]

                       # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].view(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [45]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [47]:
import time
import math
import random

N_EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 0m 24s
	Train Loss: 5.062 | Train PPL: 157.948
	Validation Loss: 4.971 | Validation PPL: 144.192
Epoch: 02 | Time: 0m 23s
	Train Loss: 4.515 | Train PPL: 91.355
	Validation Loss: 4.773 | Validation PPL: 118.295
Epoch: 03 | Time: 0m 23s
	Train Loss: 4.222 | Train PPL: 68.173
	Validation Loss: 4.561 | Validation PPL: 95.714
Epoch: 04 | Time: 0m 23s
	Train Loss: 4.003 | Train PPL: 54.778
	Validation Loss: 4.508 | Validation PPL: 90.716
Epoch: 05 | Time: 0m 23s
	Train Loss: 3.837 | Train PPL: 46.403
	Validation Loss: 4.356 | Validation PPL: 77.947
Epoch: 06 | Time: 0m 23s
	Train Loss: 3.669 | Train PPL: 39.196
	Validation Loss: 4.151 | Validation PPL: 63.489
Epoch: 07 | Time: 0m 23s
	Train Loss: 3.511 | Train PPL: 33.468
	Validation Loss: 4.083 | Validation PPL: 59.339
Epoch: 08 | Time: 0m 23s
	Train Loss: 3.379 | Train PPL: 29.341
	Validation Loss: 3.993 | Validation PPL: 54.207
Epoch: 09 | Time: 0m 23s
	Train Loss: 3.223 | Train PPL: 25.108
	Validation Loss: 3.927 | Val